<a href="https://colab.research.google.com/github/SusiGu/SemAI23/blob/main/UE04_SuGu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

## Preparation

In [114]:
# Install required packages
!pip install -q rdflib     # comment to avoid re-install with every re-run

In [105]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      


## Task 1

Sie beginnen mit einem leeren Dataset. Fügen Sie in den Default Graph Statements ein, die sagen, dass :Peter der Autor von :G1 ist, und :Mary Autor von :G2.

### Solution
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>


INSERT DATA {
     :G1 :author  :Mary.
     :G2 :author :Peter.
};

In [115]:
ds = rdflib.Dataset()

ds.parse(format="trig", data="""
prefix :     <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xrdf: <urn:x-rdflib:default> 
GRAPH xrdf:  {
     :G1 :author  :Mary.
     :G2 :author :Peter.
}
""")
print(ds.serialize(format="trig"))


@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

{
    :G1 :author :Mary .

    :G2 :author :Peter .
}




## Task 2

Schreiben Sie { :Mary :knows :Peter, :John, :Mary. } in den Named Graph :G1 und { :Peter :knows :Mary. :John :knows :Mary. } in den Named Graph :G2.


### Solution
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>



INSERT DATA { 
 GRAPH :G1 {
   :Mary :knows :Peter, :John, :Mary . 
 }
 GRAPH :G2 {
    :Peter :knows :Mary. :John :knows :Mary.
 } 
} 

In [116]:

ds.update("""
INSERT DATA { 
 GRAPH :G1 {
   :Mary :knows :Peter, :John, :Mary . 
 }
 GRAPH :G2 {
    :Peter :knows :Mary. :John :knows :Mary.
 } 
}; """) 

print(ds.serialize(format="trig"))


@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :G1 :author :Mary .

    :G2 :author :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 3

Fragen Sie mittels INSERT-WHERE die :knows-Beziehungen aus :G2 ab und fügen deren inverse :knownBy-Beziehungen in den Default-Graph ein.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

### Solution

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT { 
  ?o :knownBy ?m . 
 }
 WHERE {
    GRAPH :G2 {
      ?m :knows ?o .
    }
 }; 

In [117]:

ds.update("""
INSERT { 
   GRAPH xrdf:  {?o :knownBy ?m . }
 }
 WHERE {
    GRAPH :G2 {
      ?m :knows ?o .
    }
 }; """) 
print(ds.serialize(format="trig"))


@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :G1 :author :Mary .

    :G2 :author :Peter .

    :Mary :knownBy :John,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 4

Löschen Sie mittels DELETE-WHERE alle :knownBy-Beziehungen aus dem Default-Graph.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

### Solution

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE WHERE {?m :knownBy ?p};

In [118]:
ds.update("""
DELETE WHERE {?m :knownBy ?p};""") 

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :G1 :author :Mary .

    :G2 :author :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 5
Ermitteln Sie mittels INSERT-WHERE zu jedem Named-Graph dessen Anzahl an Statements mit der Property :knows und schreiben Sie diese in den Default Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]


### Solution 

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT 
  {?g :knowsCount ?c}
WHERE {
  GRAPH ?g {
    SELECT (COUNT(?f) AS ?c)
    WHERE 
    {
      ?p :knows ?f
     }
   GROUP BY ?g
  }
}


In [119]:
ds.update("""
INSERT {
  GRAPH xrdf:  {?g :knowsCount ?c}}
WHERE {
  GRAPH xrdf: { GRAPH ?g {
    SELECT (COUNT(?f) AS ?c)
    WHERE 
    {
      ?p :knows ?f
     }
   GROUP BY ?g
  }}
};""") 

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 6

Ermitteln Sie mittels INSERT-WHERE die Anzahl an Named Graphs und schreiben Sie sie in den Default Graph.
Ihr Update Request darf nicht enthalten: [2].

### Solution

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT {
  :ds :graphCount ?c}
WHERE {
  SELECT (COUNT(DISTINCT ?g) AS ?c) WHERE {
    GRAPH ?g { ?s ?p ?o }
  }
}

In [120]:
ds.update("""
INSERT {
  GRAPH xrdf:  {:ds :graphCount ?c}}
WHERE {
  GRAPH xrdf: { SELECT (COUNT(DISTINCT ?g) AS ?c) WHERE {
    GRAPH ?g { ?s ?p ?o }
  }}
};""") 

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :ds :graphCount 2 .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 7
Verschieben Sie mittels DELETE-INSERT-WHERE alle Metadaten zu Named Graphs (also Statements die einen Named Graph als Subjekt haben) in den entsprechenden Named Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]

### Solution
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE {
    ?g ?p ?o . 
}
INSERT {
 GRAPH ?g {
    ?g ?p ?o . 
  }
}
WHERE {
   ?g ?p ?o . 
   GRAPH ?g {}.
  
}


In [121]:
ds.update("""
DELETE {
    ?g ?p ?o . 
}
INSERT {
 GRAPH ?g {
    ?g ?p ?o . 
  }
}
WHERE {
   ?g ?p ?o . 
   GRAPH ?g {}.
  
}
;""") 
print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G1 {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :Mary :knows :John,
            :Mary,
            :Peter .
}

{
    :ds :graphCount 2 .
}

:G2 {
    :G2 :author :Peter ;
        :knowsCount 2 .

    :John :knows :Mary .

    :Peter :knows :Mary .
}




## TASK 8 
Schreiben Sie in jeden Named Graph ein Statement, dass der Autor des jeweiligen Named Graphs die :Susi kennt und aktualisieren Sie mit dem selben UpdateRequest den knowsCount.
Ihr Update Request darf nicht enthalten: [Peter, Mary]

### Solution
DELETE { 
   GRAPH ?g 
      { ?p :knows  ?o. 
      ?g :knowsCount ?c_old. } } 
   INSERT { 
   GRAPH ?g
    { ?p :knows  ?o, :Susi.
      ?g :knowsCount ?c_new.  
     } }
  WHERE {
   GRAPH ?g 
      { ?p :knows  ?o. 
        ?g :author ?p .
        ?g :knowsCount ?c_old.
        BIND(?c_old + 1 AS ?c_new)
      }
 
    
  }

In [122]:
ds.update("""
DELETE { 
   GRAPH ?g 
      { ?p :knows  ?o. 
      ?g :knowsCount ?c_old. } } 
   INSERT { 
   GRAPH ?g
    { ?p :knows  ?o, :Susi.
      ?g :knowsCount ?c_new.  
     } }
  WHERE {
   GRAPH ?g 
      { ?p :knows  ?o. 
        ?g :author ?p .
        ?g :knowsCount ?c_old.
        BIND(?c_old + 1 AS ?c_new)
      }
 
    
  };""") 

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G1 {
    :G1 :author :Mary ;
        :knowsCount 4 .

    :Mary :knows :John,
            :Mary,
            :Peter,
            :Susi .
}

{
    :ds :graphCount 2 .
}

:G2 {
    :G2 :author :Peter ;
        :knowsCount 3 .

    :John :knows :Mary .

    :Peter :knows :Mary,
            :Susi .
}


